In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification, load_iris, load_wine
from trees.Stree import Stree
import time

In [2]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [3]:
import time
from sklearn.model_selection import train_test_split
from trees.Stree import Stree

random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state, stratify=y)
    return Xtrain, Xtest, ytrain, ytest

# data = load_creditcard(-5000) # Take all true samples + 5000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
data = load_creditcard() # Take all the samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]

Fraud: 0.173% 492
Valid: 99.827% 284315
X.shape (284807, 28)  y.shape (284807,)
Fraud: 0.173% 492
Valid: 99.827% 284315


In [4]:
t = time.time()
clf = Stree(C=.01, random_state=random_state)
clf.fit(Xtrain, ytrain)
print(clf)
print(f"{time.time() - t:.4f} secs")

root
root - Down
root - Down - Down, <cgaf> - Leaf class=1 belief=0.941799 counts=(array([0, 1]), array([ 11, 178]))
root - Down - Up
root - Down - Up - Down, <pure> - Leaf class=1 belief=1.000000 counts=(array([1]), array([2]))
root - Down - Up - Up, <cgaf> - Leaf class=0 belief=0.952381 counts=(array([0, 1]), array([20,  1]))
root - Up
root - Up - Down
root - Up - Down - Down, <cgaf> - Leaf class=1 belief=0.902174 counts=(array([0, 1]), array([ 9, 83]))
root - Up - Down - Up, <pure> - Leaf class=0 belief=1.000000 counts=(array([0]), array([14]))
root - Up - Up, <cgaf> - Leaf class=0 belief=0.999598 counts=(array([0, 1]), array([198966,     80]))

42.9141 secs


In [5]:
t = time.time()
print(clf.predict(Xtest)[:17])
print(f"{time.time() - t:.4f} secs")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.2084 secs


In [6]:
t = time.time()
print(clf.predict_proba(Xtest)[:17, :])
print(f"{time.time() - t:.4f} secs")

[[0.         0.26356965]
 [0.         0.22665372]
 [0.         0.25678353]
 [0.         0.26056019]
 [0.         0.26583006]
 [0.         0.24360041]
 [0.         0.26366182]
 [0.         0.26012045]
 [0.         0.2298345 ]
 [0.         0.25726294]
 [0.         0.25909988]
 [0.         0.25940575]
 [0.         0.24256254]
 [0.         0.15094485]
 [0.         0.26327588]
 [0.         0.26382949]
 [0.         0.26290957]]
0.2083 secs


In [7]:
t = time.time()
print(f"Classifier's accuracy (train): {clf.score(Xtrain, ytrain):.4f}")
print(f"Classifier's accuracy (test) : {clf.score(Xtest, ytest):.4f}")
print(f"{time.time() - t:.4f} secs")

Classifier's accuracy (train): 0.9995
Classifier's accuracy (test) : 0.9995
0.5074 secs


In [8]:
t = time.time()
clf2 = LinearSVC(C=.01, random_state=random_state)
clf2.fit(Xtrain, ytrain)
print(clf2.score(Xtest, ytest))
print(f"{time.time() - t:.4f} secs")

0.9993211848834895
13.3835 secs


In [9]:
t = time.time()
clf3 = DecisionTreeClassifier(random_state=random_state)
clf3.fit(Xtrain, ytrain)
print(clf3.score(Xtest, ytest))
print(f"{time.time() - t:.4f} secs")

0.9991573329588147
22.7635 secs
